# Constraints Analysis

In [1]:
# Add path to src/CARPy, in case notebook is running locally
import os, sys, warnings
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..\\..\\..\\src")))
warnings.simplefilter("ignore")  # <-- Suppress warnings

***
## Introduction

CARPy provides users with the ability to constrain designs, using a flavour of Energy-manoeuvrability theory.
To make the best use of this concept, the following objects are introduced:

1. `EnergyConstraint`: A type of design constraint, based on Energy-manoeuvrability theory.
2. `Constraints`: A means of considering multiple constraints together and operating on them.
3. `Constraint`: The base constraint class that provides constraint analysis methods.

These objects are laid out in this way to facilitate future expansion of the constraint concept.

***
## 1) `EnergyConstraint` objects

The concept of an energy constraint is based on a model of aircraft performance based on Energy-manoeuvrability theory.
In short, the equations of motion are considered for a vehicle, and can then be resolved for any design variable.
CARPy's implementation makes the thrust-to-weight ratio, $\text{T/W}$, the objective.
For a detailed derivation, please see the [theory docs](https://github.com/yaseen157/carpy/tree/main/docs/source/theory).

Start with relevant imports

In [2]:
from carpy.conceptualdesign import EnergyConstraint
from carpy.environment import ISA1975

# Prescribe flight conditions at the design point
M = 0.5  # Mach number
z = 8e3  # 8,000 m altitude

# Create an atmosphere object, and use it to compute relevant parameters
myatmosphere = ISA1975()
speed = myatmosphere.c_sound(z, geometric=True) * M
dynamic_pressure = myatmosphere.rho(z, geometric=True) * speed ** 2

# Apply the computed dynamic pressure to our energy constraint
mymission = EnergyConstraint(q=dynamic_pressure)

The object instantiated from the `EnergyConstraint` class is actually an instance of `Constraints`, which allows us to consider multiple `EnergyConstraint` objects simultaneously.

***
## 2) `Constraints` objects

We can see that the resulting instance contains one `EnergyConstraint` object.

In [3]:
mymission.constraints

(<carpy.conceptualdesign._constraints.EnergyConstraint at 0x2100b7c0a00>,)

Each constraint contained with a `Constraints` object represents a single design point or goal.
But just as easily, we can revisit the design with a sequence of constraints - the design conditions are broadcasted as required.

In [4]:
M = [0.5, 0.55]  # <-- Multiple Mach numbers
z = 8e3  # <-- Single altitude is broadcasted
zdot = [1.2, 0]  # <-- Multiple climb rates

speed = myatmosphere.c_sound(z, geometric=True) * M
dynamic_pressure = myatmosphere.rho(z, geometric=True) * speed ** 2

# Apply the computed dynamic pressures to our energy constraint
mymission = EnergyConstraint(q=dynamic_pressure, zdot=zdot)

print(mymission.constraints)

(<carpy.conceptualdesign._constraints.EnergyConstraint object at 0x000002107854BE20>, <carpy.conceptualdesign._constraints.EnergyConstraint object at 0x000002100D6279A0>)


In [5]:
# Multiple, individual constraints!
constraint1, constraint2 = mymission.constraints

***
## 3) `Constraint` objects

#### Governing Equation

The governing equation for a constraint is hidden in private variable. 

In [6]:
# The governing equation of a constraint (not designed for users to interface with)
print(constraint1._eqn)

Eq(T/W, (-C_D*S*q*(mu*sin(2*alpha)/2 - 1)/W - C_L*S*mu*q*cos(alpha)**2/W + Vdot/g + mu*cos(alpha)*cos(theta) + zdot/V)/((mu*cos(alpha)*tan(alpha + epsilon) + 1)*cos(alpha + epsilon)) + (-C_D*S*mu*q*sin(alpha)**2/W - C_L*S*q*(mu*sin(2*alpha)/2 + 1)/W + Vdot_n/g + mu*cos(alpha)*cos(theta) + (1 - zdot**2/V**2)**0.5)/((mu*sin(alpha) + 1)*sin(alpha + epsilon)))


Obviously, it's not for novice users to mess with.
A slightly prettier form can be found in the base class of a constraint

In [7]:
# The governing equation for an empty energy constraint
EnergyConstraint._eqn

Eq(T/W, (-C_D*S*q*(mu*sin(2*alpha)/2 - 1)/W - C_L*S*mu*q*cos(alpha)**2/W + Vdot/g + mu*cos(alpha)*cos(theta) + zdot/V)/((mu*cos(alpha)*tan(alpha + epsilon) + 1)*cos(alpha + epsilon)) + (-C_D*S*mu*q*sin(alpha)**2/W - C_L*S*q*(mu*sin(2*alpha)/2 + 1)/W + Vdot_n/g + mu*cos(alpha)*cos(theta) + (1 - zdot**2/V**2)**0.5)/((mu*sin(alpha) + 1)*sin(alpha + epsilon)))

Or in a slightly more familiar form for cruising flight (after some substitutions):

In [8]:
import sympy as sp

# Define some symbols so we can tell the solver W = L = q * S * C_L
W, S, q, C_L = sp.symbols("W,S,q,C_L")

# The governing equation for an energy constraint
EnergyConstraint._eqn.subs({
    "epsilon":0, "mu":0, "Vdot":0, "Vdot_n":0, "zdot":0, (q * S * C_L) : W})

Eq(T/W, C_D*S*q/(W*cos(alpha)))

#### In practice

All constraint objects implement the `__call__` method.
Users can call a constraint instance, and known values are automatically substituted.

In [9]:
constraint1()

Eq(T/W, (-12477.8465402665*C_D*S*(mu*sin(2*alpha)/2 - 1)/W - 12477.8465402665*C_L*S*mu*cos(alpha)**2/W + Vdot/g + mu*cos(alpha)*cos(theta) + 1.2/V)/((mu*cos(alpha)*tan(alpha + epsilon) + 1)*cos(alpha + epsilon)) + (-12477.8465402665*C_D*S*mu*sin(alpha)**2/W - 12477.8465402665*C_L*S*(mu*sin(2*alpha)/2 + 1)/W + Vdot_n/g + mu*cos(alpha)*cos(theta) + 1.2*(0.694444444444444 - 1/V**2)**0.5)/((mu*sin(alpha) + 1)*sin(alpha + epsilon)))

Looks pretty nasty!
We can drastically simplify the expression with more substitutions

In [10]:
constraint1(
    mu=0,  # Rolling resistance of zero, for a flying vehicle
    epsilon=0,  # Thrust setting angle of zero, thrust axis aligns with the vehicle longitudinal axis
    zdot=0,  # Rate of climb zero, steady altitude (overriding our defined climb rate)
    Vdot=0,  # Acceleration along flight path is zero (no linear acceleration)
    Vdot_n=0  # Acceleration normal to flight path is zero (no circular motion)
)

Eq(T/W, 12477.8465402665*C_D*S/(W*cos(alpha)) + (-12477.8465402665*C_L*S/W + 1)/sin(alpha))

That's better.
In this case the angle of attack $\alpha$, coefficients of lift $C_L$ and drag $C_D$, and wing-loading are all closely related and very inter-dependent parameters.
Let's make some more substitutions:

In [11]:
# Make the earlier substitutions stick for future evaluations
constraint1.mu = 0
constraint1.epsilon = 0
constraint1.zdot = 0
constraint1.Vdot = 0
constraint1.Vdot_n = 0

# Take it one step further
constraint1.alpha=(1 * 3.141592/180)  # 1 degree angle of attack

# Show the equation in the notebook
constraint1()

Eq(T/W, 12479.7472654312*C_D*S/W - 714964.390772459*C_L*S/W + 57.2987004179959)

In some cases such as this, $W$ or $S$ may not be individually known, however, the ratios $\text{T/W}$ and $\text{W/S}$ can be solved and commuted

In [12]:
import sympy as sp
T, W, S = sp.symbols("T,W,S")

# Define the wingloading ratio
wingloading = {W: 2000 * S}

# Solve for T/W, then substitute in wingloading
eqn_t2w = sp.solve(constraint1(), T/W)[0]
eqn_t2w = eqn_t2w.subs(wingloading).simplify()

# Show the equation in the notebook
print(f"T/W = {eqn_t2w}")

T/W = 6.2398736327156*C_D - 357.48219538623*C_L + 57.2987004179959


In [13]:
# Parameters below come from guessing numbers that made a nice T/W.
# In a real-world scenario, a solver paired with an aerodynamic model is better.
lift_to_drag = 12
CL = 0.16

# Make the substitutions using a dictionary
t2w = eqn_t2w.subs({"C_L":CL, "C_D":CL / lift_to_drag})

# In this case, our desired value is on the right hand side of the governing equation equality
print(f"The computed Thrust-to-Weight ratio required at the design point is: T/W = {t2w:.3f}")

The computed Thrust-to-Weight ratio required at the design point is: T/W = 0.185
